<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#4.-Inference" data-toc-modified-id="4.-Inference-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>4. Inference</a></span><ul class="toc-item"><li><span><a href="#4-1.-(just-for-test)-Data-Preparation" data-toc-modified-id="4-1.-(just-for-test)-Data-Preparation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>4-1. (just for test) Data Preparation</a></span></li></ul></li></ul></div>

# 4. Inference

In [1]:
import sys, os
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
import setting

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.common.common import p2_dataSelection as p2
from Clust.clust.ML.common.common import p4_testing as p4
from Clust.clust.ML.classification import app
import pathSetting


import torch
#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


## 4-1. (just for test) Data Preparation 

In [2]:
# 0. pick only one data
DataMeta = p1.readJsonData(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())
# dataX

dataName_X = dataList[2]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[3]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)

window_num =128
data_X = p2.getSavedIntegratedData(dataSaveMode_X, dataName_X, dataFolderPath)[:window_num]
# datay = p2.getSavedIntegratedData(dataSaveMode_y, dataName_y, dataFolderPath)[:1]
# from Clust.clust.transformation.type.DFToNPArray import transDFtoNP
# inputX, inputy = transDFtoNP(dataX, datay, windowNum)
# print(inputX, inputy) # Real Input

### Get model meta by mongodb

In [3]:
from Clust.clust.ingestion.mongo import mongoClient as mongo_client
from Clust.setting import influx_setting_KETI as ins

mongo_client = mongo_client.mongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)

In [4]:
search = {'modelName': 'ClassificationNocleanLSTM_cf'}
model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)
model_meta = model_meta[0]

In [5]:
model_meta

{'author': 'KETI',
 'modelName': 'ClassificationNocleanLSTM_cf',
 'modelVersion': 0.1,
 'trainDataInfo': {'startTime': '1992-1-11',
  'endTime': '2012-02-27',
  'dataInfo': [['bio_action_sensors', 'accelerationTrain']],
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}},
  'integration_freq_sec': 675,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'featureList': ['col_0',
  'col_1',
  'col_2',
  'col_3',
  'col_4',
  'col_5',
  'col_6',
  'col_7',
  'col_8'],
 'target': ['value'],
 'trainDataType': 'timeseries',
 'modelPurpose': 'classification',
 'model_method': 'LSTM_cf',
 'modelTags': ['action', 'sensor', 'classification', 'pattern'],
 'cleanTrainDataParam': 'NoClean',
 'NaNProcessing

In [6]:
df_result = app.get_inference_result(data_X, model_meta, window_num)

noScale

Start testing data

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[4]
@!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@2
[4]
** Dimension of result for inference dataset = (1,)
['value']
[4]
<class 'numpy.ndarray'>


In [7]:
df_result

,value
0,4
